In [191]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
from keras.layers import Dense, Bidirectional, LSTM, Input, Embedding
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
import os
import numpy as np
CHAR_LEN = 10
PHONEME_LEN = 6
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 22

In [198]:
authors = json.loads(requests.get(url="http://poetrydb.org/author").content)

ConnectionError: HTTPConnectionPool(host='poetrydb.org', port=80): Max retries exceeded with url: /author (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a6ed75050>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [119]:
embeddings_index = {}
f = open('../glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [189]:
arpabet = nltk.corpus.cmudict.dict()

def scrape_author(author):
    return pd.DataFrame(json.loads(requests.get(url="http://poetrydb.org/author/{}/author,title,lines".format(author)).content))

def generate_phonemes(word):
    try:
        phonemes = arpabet[word]
    except:
        phonemes = []
    if len(phonemes[0]) > PHONEME_LEN:
        raise Exception(phonemes)
    phonemes = phonemes[0] + [u''] * (PHONEME_LEN - len(phonemes[0]))

    return phonemes[0:PHONEME_LEN]

def generate_characters(word):
    chars = list(word) + [u''] * (CHAR_LEN - len(word))
    return chars

In [190]:
poems = scrape_author(authors["authors"][1])

ConnectionError: HTTPConnectionPool(host='poetrydb.org', port=80): Max retries exceeded with url: /author/Alan%20Seeger/author,title,lines (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a6edda450>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [168]:
s = poems.apply(lambda x: pd.Series(x['lines']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'line'
poems_expanded = poems.drop('lines', axis=1).join(s)


In [169]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems_expanded.line)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 4313 unique tokens.


In [174]:
sequences = tokenizer.texts_to_sequences(poems_expanded.line)


In [148]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [170]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [178]:
sequences = [x for x in sequences if len(x) > 2]

In [181]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [184]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [185]:
embedding_layer

In [92]:
# words
# characters
# phonemes
# document id
# author id
CHAR_LEN = 10
PHONEME_LEN = 10

words = Input()
chars = Input()
phonemes = Input()
doc = Input()
auth = Input()

word_embed = Embedding()(words)
char_lstm = Bidirectional(LSTM(CHAR_LEN, return_sequences=False), input_shape=(5, CHAR_LEN))(chars)
phoneme_lstm = Bidirectional(LSTM(PHONEME_LEN, return_sequences=False), input_shape=(5, PHONEME_LEN))(phonemes)





x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training

AssertionError: Please provide to Input either a `shape` or a `batch_shape` argument. Note that `shape` does not include the batch dimension.